In [2]:

import numpy as np
import palettable 
import numpy as np
from Satellite import *
from Simulator import *

print("Generating Satellites...")
sat1 = Satellite("Satellites/sat1.txt", "SAT1")
sat2 = Satellite("Satellites/sat2.txt", "SAT2")
sat3 = Satellite("Satellites/sat3.txt", "SAT3")
sat4 = Satellite("Satellites/sat4.txt", "SAT4")
sat5 = Satellite("Satellites/sat5.txt", "SAT5")
sat6 = Satellite("Satellites/sat6.txt", "SAT6")  
sat7 = Satellite("Satellites/sat7.txt", "SAT7")
sat8 = Satellite("Satellites/sat8.txt", "SAT8")
sat9 = Satellite("Satellites/sat9.txt", "SAT9")  
sat10 = Satellite("Satellites/sat10.txt", "SAT10")
sat11 = Satellite("Satellites/sat11.txt", "SAT11")
sat12 = Satellite("Satellites/sat12.txt", "SAT12")
nav1 = navSatellite("navSatellites/GNSS1.txt", "GNSS1")
nav2 = navSatellite("navSatellites/GNSS2.txt", "GNSS2")
nav3 = navSatellite("navSatellites/GNSS3.txt", "GNSS3")
nav4 = navSatellite("navSatellites/GNSS4.txt", "GNSS4")
gs = groundStation(-32.9986, 148.2621, 415)
print("Satellites created")
sim = Simulator([sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12], [nav1,nav2,nav3,nav4], [gs])
sim.simulate(0,6*60*60,1, motionEquation)
mappingArray= [] 


Generating Satellites...


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
import geemap
import folium
import ee
import os
service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)



count = 0
import csv
"""
for sat in sim.satellites:
    csv_file = "RESULTS_"+sat.name
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)  
        i = 0      
        for image in  sat.payload.images:
            coord1 = list(image[0][0:2])
            coord2 = list(image[1][0:2])
            coord3 = list(image[2][0:2])
            coord4 = list(image[3][0:2])
            coord1.reverse()
            coord2.reverse()
            coord3.reverse()
            coord4.reverse()

            data = [coord1[0], coord1[1],  coord4[0], coord4[1], sat.payload.mappingErrors[i]]
        
            writer.writerow(data)
            roi = ee.Geometry.Rectangle([[coord1[0], coord1[1]],  [coord4[0], coord4[1]]])
            newFeature = ee.Feature(roi, {'name': str(count)})
            count = count+1
            mappingArray.append(newFeature)
            i+=1
"""

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)
mappedArea = ee.FeatureCollection(mappingArray)
NSW = ee.Geometry.Rectangle([[144.81577841465915, -36.42295520530404], [155.81727106335805, -27.806830550620564]])
# Create an image collection for Sentinel-2 data
start_date = '2019-06-01'
end_date = '2020-05-31'
sentinel2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filterBounds(mappedArea)
                        .select('B.*|MSK_CLDPRB|SCL')
)
burnedArea = ee.Image('projects/jimssatellites/assets/fires')
nir = sentinel2_collection.select('B8')  # Near-Infrared Band (B8)
swir1 = sentinel2_collection.select('B11')  # Short-Wave Infrared Band 1 (B11)
swir2 = sentinel2_collection.select('B12')  # Short-Wave Infrared Band 2 (B12)


##CREATE THE BLACK MASK
"""
roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)
fc = ee.FeatureCollection(mappingArray)
# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))
"""





nir_palette = ['000000', '330000', '660000', '990000', 'CC0000', 'FF0000', 'FF1900', 'FF3200', 'FF4B00', 'FF6400',
              'FF7D00', 'FF9600', 'FFAF00', 'FFC800', 'FFE100', 'FFFF00']
# Create a visualization palette
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}
# Define visualization parameters for each band
nir_vis_params = {
    'min': 0,
    'max': 10000,
    'palette': nir_palette  # Apply the custom color palette for NIR
}
#Map.addLayer(sentinel2_collection, vis_params, 'Sentinel-2 Image')
#Map.addLayer(nir, nir_vis_params, 'NIR')

#Short wave infrared
Map.addLayer(swir1, nir_vis_params, 'SWIR1')


#Burned Area
Map.addLayer(burnedArea)

#Black mask
Map.addLayer(result, {
    'palette': '000000,FF0000',
    'max': 1,
    'opacity': 1.0
}, 'Opaque White Rectangle')


#Map.addLayer(swir1, {'min': 0, 'max': 3000}, 'SWIR1')
Map


NameError: name 'mappingArray' is not defined